In [1]:
class Environment:
    def __init__(self):
        self.steps = 10
        self.board = [False for x in range(0, 5)]
        self.board[4] = True

    def reset(self):
        self.steps = 10

    def start(self):
        return 0

    def end(self):
        return len(self.board)

    def is_done(self) -> bool:
        return self.steps == 0

    def action(self, state) -> bool:
        self.steps -= 1
        if self.is_done():
            print("Did not reach the goal")
        return self.board[state]


In [4]:
import random

class Agent:
    def __init__(self):
        self.state = 0
        self.actions = [0,1]

    def reset(self):
        self.state = 0

    def forward(self, limit):
        self.state = min(self.state + 1, limit-1)

    def back(self, lowLimit):
        self.state = max(self.state - 1, lowLimit)

    def action(self, env: Environment):
        action = random.choice(self.actions)
        if action:
            self.forward(env.end())
        else:
            self.back(env.start())
        return env.action(self.state)


In [5]:
def main():
    episodes = 10
    a = Agent()
    e = Environment()
    for i in range(0, episodes):
        print(f"Episode: {i+1}")
        count = 0
        while not e.is_done():
            count += a.action(e)
        print(f"{count} success - {episodes - count} Fail")
        a.reset()
        e.reset()

main()

Episode: 1
Did not reach the goal
0 success - 10 Fail
Episode: 2
Did not reach the goal
1 success - 9 Fail
Episode: 3
Did not reach the goal
0 success - 10 Fail
Episode: 4
Did not reach the goal
0 success - 10 Fail
Episode: 5
Did not reach the goal
0 success - 10 Fail
Episode: 6
Did not reach the goal
0 success - 10 Fail
Episode: 7
Did not reach the goal
0 success - 10 Fail
Episode: 8
Did not reach the goal
0 success - 10 Fail
Episode: 9
Did not reach the goal
0 success - 10 Fail
Episode: 10
Did not reach the goal
0 success - 10 Fail


In [7]:
#Paso 1. Definir la clase Learner
import random

class Learner:
    def __init__(self, agent, env, alpha=0.1, gamma=0.6, epsilon=0.1):
        self.agent = agent
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.table = self.__inittable__()

    def __inittable__(self):
        # Inicializar la tabla con ceros. La dimensión es el número de estados x número de acciones.
        # Como hay 5 posiciones (estados) y 2 acciones (avanzar, retroceder), la tabla será de 5x2.
        return np.zeros((self.env.end(), len(self.agent.actions)))


#Paso 2. Incluyendo la ejecucion:
    def run(self):
        done = False
        while not done:
            current_state = self.agent.state
            # Decidir entre tomar una acción al azar o la mejor acción basada en epsilon
            if random.uniform(0, 1) < self.epsilon:

     #Paso 3. Escogencia de acción aleatoria:

                action = random.choice(self.agent.actions)
            else:
                # Mejor acción basada en la tabla de aprendizaje
                action = np.argmax(self.table[current_state])

            # Realizar la acción y obtener el nuevo estado, recompensa, y si se terminó
            reward = self.env.action(current_state)
            done = self.env.is_done()
            next_state = self.agent.state

            # Calcular el valor actual y el siguiente mejor valor
            old_value = self.table[current_state, action]
            next_max = np.max(self.table[next_state])

            # Calcular el nuevo valor
            new_value = (1 - self.alpha) * old_value + self.alpha * (reward + self.gamma * next_max)

            # Actualizar la tabla
            self.table[current_state, action] = new_value

            if not done:
                # Actualizar el estado del agente si no se ha terminado
                if action == 1:
                    self.agent.forward(self.env.end())
                else:
                    self.agent.back(self.env.start())

    def step(self, action):
          # Calcular el estado actual del agente
          old_state = self.agent.state

          # Obtener la recompensa para el estado actual y la acción tomada
          reward = self.getReward(old_state, action)

          # Avanzar el estado del agente ejecutando la acción tomada
          if action == 0:
              self.agent.back(self.env.start())
          else:
              self.agent.forward(self.env.end())

          # Calcula el nuevo estado del agente
          new_state = self.agent.state

          # Retorna el viejo estado, la recompensa, y el nuevo estado
          return old_state, reward, new_state

    def getReward(self, state, action):
        # Define la recompensa base como 0
        reward = 0
        # Define si el episodio ha terminado
        done = False
        if state == 4:
            reward = 10
            done = True
        else:
            pass

        return reward, done


In [8]:
class Learner:
    # Asumir que el resto de la clase Learner ya está definido anteriormente

    def step(self, action):
        # Calcula el estado actual del agente
        old_state = self.agent.state

        # Obtener la recompensa para el estado actual y la acción tomada
        reward = self.getReward(old_state, action)

        # Avanza el estado del agente ejecutando la acción tomada
        # Esto puede variar según cómo esté implementado tu agente.
        # Por ejemplo, si tu agente tiene métodos forward y back:
        if action == 0:  # Suponiendo que 0 representa retroceder y 1 avanzar
            self.agent.back(self.env.start())
        else:
            self.agent.forward(self.env.end())

        # Calcula el nuevo estado del agente
        new_state = self.agent.state

        # Retorna el viejo estado, la recompensa, y el nuevo estado
        return old_state, reward, new_state

    def getReward(self, state, action):
        # Esta función necesita ser implementada para calcular la recompensa
        # basada en el estado y la acción. Esto es solo un placeholder.
        # Necesitarás implementar la lógica específica para obtener la recompensa.
        return 0  # Placeholder para la recompensa


In [9]:
class Learner:
    # Asumir que el resto de la clase Learner ya está definido anteriormente

    def getReward(self, state, action):
        # Define la recompensa base como 0
        reward = 0
        # Define si el episodio ha terminado
        done = False

        # Si el agente alcanza la posición más a la derecha del tablero (estado 4),
        # se le recompensa con 10 y se termina el episodio
        if state == 4:
            reward = 10
            done = True
        else:
            # Si no, puedes definir otras recompensas para diferentes estados/actions aquí
            # Por ejemplo, penalizaciones o recompensas menores por aproximarse al objetivo
            pass

        return reward, done


In [10]:
class Environment:
    def __init__(self):
        self.board = [False for x in range(0, 5)]
        self.board[4] = True

    def start(self):
        return 0

    def end(self):
        return len(self.board)


In [14]:
class Agent:
    def __init__(self):
        self.state = 0
        self.actions = [0, 1]  # 0 para "back", 1 para "forward"
        self.rightBound = rightBound  # Tamaño del tablero

    def reset(self):
        self.state = 0

    def forward(self):
        # Modificado para no recibir parámetros; usa rightBound para limitar el estado
        self.state = min(self.state + 1, self.rightBound - 1)

    def back(self):
        # Modificado para no recibir parámetros
        self.state = max(self.state - 1, 0)

    def action(self, action_id):
        # Modificado para recibir un identificador de acción
        if action_id == 1:
            self.forward()
        else:
            self.back()

    def getAction(self, action_number):
        # Retorna el nombre de la acción basado en el número de acción
        return "forward" if action_number == 1 else "back"


In [15]:
# Asegúrate de que las clases Agent, Environment y Learner estén definidas correctamente
# según las instrucciones anteriores

def main():
    episodes = 10
    e = Environment()  # Asumiendo que el ambiente no necesita parámetros o están ya configurados
    a = Agent()  # Pasamos el tamaño del tablero como el límite derecho
    learner = Learner(agent=a, env=e)  # Crea el objeto Learner con el agente y el ambiente

    for i in range(episodes):
        print(f"Episode: {i+1}")
        learner.run()  # Ejecuta el loop de aprendizaje del agente mediante Learner
        a.reset()  # Solo reseteamos el estado del agente para el próximo episodio

main()


NameError: name 'rightBound' is not defined